In [1]:
# IMPORT LIBRARY
import os
import time
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# PARAMETER GLOBAL & SEED
FINAL_DATASET = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"

CLASSES = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# DATA GENERATOR (RESCALE ONLY)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "test"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 2001 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [4]:
# MODEL MOBILENETV2
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze feature extractor
base_model.trainable = False

In [5]:
# Build model (APPLE TO APPLE dengan V1)
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation="relu"),
    Dense(len(CLASSES), activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          81,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,340,358 (8.93 MB)

 Trainable params: 82,374 (321.77 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
# EARLY STOPPING
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [7]:
# TRAINING + TIME MEASUREMENT
start_time = time.time()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()

total_training_time = end_time - start_time
epochs_ran = len(history.history["loss"])

print(f"\nTotal Training Time : {total_training_time:.2f} seconds")
print(f"Average / Epoch    : {total_training_time / epochs_ran:.2f} seconds")
print(f"Training stopped at epoch : {epochs_ran}")


Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 263ms/step - accuracy: 0.3783 - loss: 1.6324 - val_accuracy: 0.5836 - val_loss: 1.2298
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 247ms/step - accuracy: 0.6102 - loss: 1.1155 - val_accuracy: 0.6844 - val_loss: 0.9513
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 261ms/step - accuracy: 0.7011 - loss: 0.8785 - val_accuracy: 0.7135 - val_loss: 0.8225
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 265ms/step - accuracy: 0.7491 - loss: 0.7421 - val_accuracy: 0.7241 - val_loss: 0.7451
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 260ms/step - accuracy: 0.7766 - loss: 0.6565 - val_accuracy: 0.7480 - val_loss: 0.7086
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 257ms/step - accuracy: 0.8011 - loss: 0.5901 - val_accuracy: 0.7586 - val_loss: 0.6571
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 260ms/step - accuracy: 0.8246 - loss: 0.5359 - val_accuracy: 0.7692 - val_loss: 0.6422
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 266ms/step - accuracy: 0.8386 - loss: 0.4913 - val_accu

In [8]:
# Waktu Inferensi Keseluruhan Test Set
start_time = time.time()

predictions = model.predict(test_data)

end_time = time.time()

inference_time_total = end_time - start_time
num_samples = test_data.samples

print(f"Total Inference Time (Test Set): {inference_time_total:.4f} seconds")
print(f"Average Inference Time per Image: {inference_time_total / num_samples:.6f} seconds")

12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step 
Total Inference Time (Test Set): 4.0858 seconds
Average Inference Time per Image: 0.010668 seconds


In [9]:
# Evaluasi Model (Test Set – Tetap Sama)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - accuracy: 0.8251 - loss: 0.5355
Test Accuracy: 0.8251


In [10]:
# Ambil 1 batch
x_batch, _ = next(test_data)

# Warm-up (penting untuk CNN)
_ = model.predict(x_batch[:1])

start_time = time.time()
_ = model.predict(x_batch[:1])
end_time = time.time()

print(f"Inference Time (Single Image): {(end_time - start_time):.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Inference Time (Single Image): 0.064460 seconds


In [11]:
# CONFUSION MATRIX & REPORT
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 223ms/step

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.96      0.84      0.89        61
       glass       0.78      0.83      0.80        76
       metal       0.91      0.85      0.88        62
       paper       0.86      0.93      0.89        90
     plastic       0.74      0.74      0.74        72
       trash       0.57      0.55      0.56        22

    accuracy                           0.83       383
   macro avg       0.80      0.79      0.79       383
weighted avg       0.83      0.83      0.83       383


Confusion Matrix:
[[51  1  0  6  1  2]
 [ 0 63  1  1 10  1]
 [ 0  2 53  4  3  0]
 [ 2  0  0 84  2  2]
 [ 0 12  2  1 53  4]
 [ 0  3  2  2  3 12]]


In [13]:
# SAVE MODEL (KERAS FORMAT)
MODEL_PATH = "mobilenetv2_trashnet_PreInput.keras"
model.save(MODEL_PATH)

print(f"\nModel berhasil disimpan di: {MODEL_PATH}")


Model berhasil disimpan di: mobilenetv2_trashnet_PreInput.keras
